In [8]:
train_dir = '../data/stretched/train'
# test_dir = '../data/stretched/test'
validation_dir = '../data/stretched/val'

img_shape = (96, 96, 3)

def get_model():
    from keras.applications import VGG16
    from tensorflow.python.keras.models import Sequential
    from tensorflow.python.keras.layers import Flatten, Dropout, Dense

    conv_base = VGG16(weights='imagenet', include_top=False, input_shape= img_shape)
    conv_base.trainable = False

    model = Sequential()
    model.add(conv_base)
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(7, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

    return model

In [9]:

def train():
    from keras_preprocessing.image import ImageDataGenerator
    from keras.applications.vgg16 import preprocess_input
    import keras.callbacks as clb

    callbacks = [
            clb.ReduceLROnPlateau(monitor='val_acc', factor=0.5, min_lr=1e-7, patience=2, verbose=1),
            clb.EarlyStopping(monitor='val_acc', patience=6, verbose=1),
            clb.ModelCheckpoint(monitor='val_acc', filepath='../models/external_fast_model.h5',
                                save_best_only=True, verbose=1)
            ]
    model = get_model()
    model.summary()

    img_batch_size = 128
    data_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
    train_gen = data_gen.flow_from_directory(train_dir, target_size=img_shape[:2], batch_size=img_batch_size)
    val_gen = data_gen.flow_from_directory(validation_dir, target_size=img_shape[:2], batch_size=img_batch_size)
    nr_of_train_pictures = 28709
    train_steps = nr_of_train_pictures // img_batch_size + 1
    nr_of_valid_pictures = 3589
    valid_steps = nr_of_valid_pictures // img_batch_size + 1
    hist = model.fit(train_gen, steps_per_epoch=train_steps, epochs=100,
                        validation_data=val_gen, validation_steps=valid_steps, callbacks=callbacks,
                        verbose=2
                        )
    return hist.history

history = train()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 3, 3, 512)         14714688  
_________________________________________________________________
flatten_3 (Flatten)          (None, 4608)              0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 4608)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 512)               2359808   
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 128)               65664     
_________________________________________________________________
dense_11 (Dense)             (None, 7)                

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

epochs = range(len(history['acc']))
plt.plot(epochs, history['acc'], label='Accuracy')
plt.plot(epochs, history['val_acc'], '.', label='Validation accuracy')
plt.legend()
plt.show()

plt.figure()
plt.plot(epochs, history['loss'], label='loss')
plt.plot(epochs, history['val_loss'], label='validation_loss')